# Model Save/Load (TF2)

https://www.tensorflow.org/tutorials/keras/save_and_load?hl=ko

In [1]:
# 설치
!pip install -q --upgrade pip
!pip install -q pyyaml h5py  # HDF5 포맷으로 모델을 저장하기 위해서 필요합니다

In [2]:
# 경고 메시지 출력 끄기
import warnings 
warnings.filterwarnings(action='ignore')

# 노트북 셀 표시를 브라우저 전체 폭 사용하기
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.display import clear_output

%matplotlib inline
import matplotlib.pyplot as plt

import os, sys, shutil

rseed = 22
import random
random.seed(rseed)

import numpy as np
np.random.seed(rseed)
np.set_printoptions(precision=5)
np.set_printoptions(formatter={'float_kind': "{:.5f}".format})

import pandas as pd
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format = '{:,.5f}'.format

import tensorflow as tf
tf.random.set_seed(rseed)
tf.keras.backend.set_floatx('float32') # keras default float type 설정

import tensorflow_hub as tfhub
import tensorflow_datasets as tfds

import seaborn as sns

print(f"python ver={sys.version}")
print(f"pandas ver={pd.__version__}")
print(f"numpy ver={np.__version__}")
print(f"tensorflow ver={tf.__version__}")
print(f"tensorflow execuring eagerly={tf.executing_eagerly()}")
print(f"tensorflow hub ver={tfhub.__version__}")
print(f"tensorflow GPU={'True' if tf.config.experimental.list_physical_devices('GPU') else 'False'}")
print(f"senborn ver={sns.__version__}")

python ver=3.7.6 (default, Nov 21 2020, 22:51:13) 
[Clang 12.0.0 (clang-1200.0.32.27)]
pandas ver=1.0.5
numpy ver=1.19.5
tensorflow ver=2.4.1
tensorflow execuring eagerly=True
tensorflow hub ver=0.11.0
tensorflow GPU=False
senborn ver=0.10.0


## 저장 방식

사용하는 API에 따라서 여러가지 방법으로 텐서플로 모델을 저장할 수 있습니다. 이 문서는 텐서플로 모델을 만들고 훈련하기 위한 고수준 API인 tf.keras를 사용합니다. 다른 방법들에 대해서는 텐서플로의 [저장과 복원 문서](https://www.tensorflow.org/guide/saved_model?hl=ko)와 [즉시 실행(eager execution) 문서의 저장하기 섹션](https://www.tensorflow.org/guide/eager?hl=ko#object-based_saving)을 참고하세요.


## 예제 데이터셋 받기

MNIST 데이터셋으로 모델을 훈련하여 가중치를 저장하는 예제를 만들어 보겠습니다. 모델 실행 속도를 빠르게 하기 위해 샘플에서 처음 1,000개만 사용겠습니다:

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [4]:
# 간단한 Sequential 모델을 정의합니다
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10)
    ])
    
    model.compile(optimizer='adam', 
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    return model

# 모델 객체를 만듭니다
model = create_model()

# 모델 구조를 출력합니다
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## 훈련하는 동안 체크포인트 저장하기

훈련 중간과 훈련 마지막에 체크포인트(checkpoint)를 자동으로 저장하도록 하는 것이 많이 사용하는 방법입니다. 다시 훈련하지 않고 모델을 재사용하거나 훈련 과정이 중지된 경우 이어서 훈련을 진행할 수 있습니다. tf.keras.callbacks.ModelCheckpoint은 이런 작업을 수행하는 콜백(callback)입니다. 이 콜백은 체크포인트 작업을 조정할 수 있도록 여러가지 매개변수를 제공합니다.

### 체크포인트 콜백 사용하기

훈련하는 동안 가중치를 저장하기 위해 ModelCheckpoint 콜백을 만들어 보죠:

In [5]:
checkpoint_path = "/tmp/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
print(f"checkpoint_dir: {checkpoint_dir}")
if os.path.exists(checkpoint_dir) and os.path.isdir(checkpoint_dir):
    shutil.rmtree(checkpoint_dir)

# 모델의 가중치를 저장하는 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True, verbose=1)

# 새로운 콜백으로 모델 훈련하기
model.fit(train_images, train_labels, epochs=10,
          validation_data=(test_images,test_labels), callbacks=[cp_callback])  # 콜백을 훈련에 전달합니다

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

checkpoint_dir: /tmp/training_1
Epoch 1/10
32/32 [==============================] - 1s 30ms/step - loss: 1.6218 - accuracy: 0.4786 - val_loss: 0.7245 - val_accuracy: 0.7790

Epoch 00001: saving model to /tmp/training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 3ms/step - loss: 0.4634 - accuracy: 0.8811 - val_loss: 0.5206 - val_accuracy: 0.8440

Epoch 00002: saving model to /tmp/training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 3ms/step - loss: 0.2807 - accuracy: 0.9353 - val_loss: 0.4686 - val_accuracy: 0.8540

Epoch 00003: saving model to /tmp/training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 3ms/step - loss: 0.2214 - accuracy: 0.9441 - val_loss: 0.4320 - val_accuracy: 0.8660

Epoch 00004: saving model to /tmp/training_1/cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 3ms/step - loss: 0.1763 - accuracy: 0.9547 - val_loss: 0.4132 - val_accuracy: 0.8650

Epoch 00005: saving model to /tmp/training_1/cp.ck

이 코드는 텐서플로 체크포인트 파일을 만들고 에포크가 종료될 때마다 업데이트합니다:

In [6]:
!echo "PATH={checkpoint_dir}"
!ls -al {checkpoint_dir}

PATH=/tmp/training_1
total 9568
drwxr-xr-x   5 comafire  wheel      160  3  7 00:06 .
drwxrwxrwt  17 root      wheel      544  3  7 00:06 ..
-rw-r--r--   1 comafire  wheel       71  3  7 00:06 checkpoint
-rw-r--r--   1 comafire  wheel  4886673  3  7 00:06 cp.ckpt.data-00000-of-00001
-rw-r--r--   1 comafire  wheel     1222  3  7 00:06 cp.ckpt.index


훈련하지 않은 새로운 모델을 만들어 보겠습니다. 가중치만 복원할 땐 원본 모델과 동일한 구조로 모델을 만들어야 합니다. 여기서는 동일한 구조로 모델을 만들었으므로 다른 객체이지만 가중치를 공유할 수 있습니다.

훈련하지 않은 새 모델을 만들고 테스트 세트에서 평가해 보죠. 훈련되지 않은 모델의 성능은 무작위로 선택하는 정도의 수준입니다(~10% 정확도):

In [7]:
# 기본 모델 객체를 만듭니다
model = create_model()

# 모델을 평가합니다
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3566 - accuracy: 0.1520
훈련되지 않은 모델의 정확도: 15.20%


체크포인트에서 가중치를 로드하고 다시 평가해 보죠:

In [8]:
# 가중치 로드
model.load_weights(checkpoint_path)

# 모델 재평가
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4083 - accuracy: 0.8720
복원된 모델의 정확도: 87.20%


### 체크포인트 콜백 매개변수

체크포인트 이름을 고유하게 만들어 저장할 수 있습니다. 에포크 횟수가 많아지면 저장되는 모델이 많아지기 때문에 이전 에포크보다 향상될때만 저장해 봅니다. 

새로운 모델을 훈련하고 에포크마다 고유한 이름으로 Best 체크포인트만을 저장해 보겠습니다: (기존 예제에서 사용되었던 period 파라미터는 deprecated 됨) 

(노트: 텐서플로는 따로 설정하지 않는다면 기본적으로 최근 5개의 체크포인트만 저장합니다.)

In [9]:
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "/tmp/training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
print(f"checkpoint_dir: {checkpoint_dir}")
if os.path.exists(checkpoint_dir) and os.path.isdir(checkpoint_dir):
    shutil.rmtree(checkpoint_dir)
    
# 에포크마다 Best 가중치를 저장하기 위한 콜백을 만듭니다
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, verbose=1, 
                                                 save_weights_only=True, save_best_only=True) 

# 새로운 모델 객체를 만듭니다
model = create_model()

# `checkpoint_path` 포맷을 사용하는 가중치를 저장합니다
model.save_weights(checkpoint_path.format(epoch=0))

# 새로운 콜백을 사용하여 모델을 훈련합니다
model.fit(train_images, train_labels, epochs=50, 
          callbacks=[cp_callback], validation_data=(test_images,test_labels), verbose=0)

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

checkpoint_dir: /tmp/training_2

Epoch 00001: val_loss improved from inf to 0.73039, saving model to /tmp/training_2/cp-0001.ckpt

Epoch 00002: val_loss improved from 0.73039 to 0.53101, saving model to /tmp/training_2/cp-0002.ckpt

Epoch 00003: val_loss improved from 0.53101 to 0.46557, saving model to /tmp/training_2/cp-0003.ckpt

Epoch 00004: val_loss improved from 0.46557 to 0.42657, saving model to /tmp/training_2/cp-0004.ckpt

Epoch 00005: val_loss improved from 0.42657 to 0.41130, saving model to /tmp/training_2/cp-0005.ckpt

Epoch 00006: val_loss did not improve from 0.41130

Epoch 00007: val_loss improved from 0.41130 to 0.37906, saving model to /tmp/training_2/cp-0007.ckpt

Epoch 00008: val_loss did not improve from 0.37906

Epoch 00009: val_loss did not improve from 0.37906

Epoch 00010: val_loss did not improve from 0.37906

Epoch 00011: val_loss did not improve from 0.37906

Epoch 00012: val_loss did not improve from 0.37906

Epoch 00013: val_loss did not improve from 0.37

In [10]:
!echo "PATH={checkpoint_dir}"
!ls -al {checkpoint_dir}

PATH=/tmp/training_2
total 60560
drwxr-xr-x  17 comafire  wheel      544  3  7 00:06 .
drwxrwxrwt  17 root      wheel      544  3  7 00:06 ..
-rw-r--r--   1 comafire  wheel       81  3  7 00:06 checkpoint
-rw-r--r--   1 comafire  wheel  1628726  3  7 00:06 cp-0000.ckpt.data-00000-of-00001
-rw-r--r--   1 comafire  wheel      402  3  7 00:06 cp-0000.ckpt.index
-rw-r--r--   1 comafire  wheel  4886685  3  7 00:06 cp-0001.ckpt.data-00000-of-00001
-rw-r--r--   1 comafire  wheel     1222  3  7 00:06 cp-0001.ckpt.index
-rw-r--r--   1 comafire  wheel  4886685  3  7 00:06 cp-0002.ckpt.data-00000-of-00001
-rw-r--r--   1 comafire  wheel     1222  3  7 00:06 cp-0002.ckpt.index
-rw-r--r--   1 comafire  wheel  4886685  3  7 00:06 cp-0003.ckpt.data-00000-of-00001
-rw-r--r--   1 comafire  wheel     1222  3  7 00:06 cp-0003.ckpt.index
-rw-r--r--   1 comafire  wheel  4886685  3  7 00:06 cp-0004.ckpt.data-00000-of-00001
-rw-r--r--   1 comafire  wheel     1222  3  7 00:06 cp-0004.ckpt.index
-rw-r--r--   1 

In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'/tmp/training_2/cp-0007.ckpt'

모델을 초기화하고 최근 체크포인트를 로드하여 테스트해 보겠습니다:

In [12]:
# 새로운 모델 객체를 만듭니다
model = create_model()

# 이전에 저장한 가중치를 로드합니다
model.load_weights(latest)

# 모델을 재평가합니다
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.3791 - accuracy: 0.8740
복원된 모델의 정확도: 87.40%


### 수동으로 가중치 저장하기

앞에서 가중치를 모델에 로드하는 방법을 보았습니다. 수동으로 가중치를 저장하는 것도 쉽습니다. [Model.save_weights 메서드](https://www.tensorflow.org/api_docs/python/tf/keras/Model?hl=ko#save_weights)를 사용합니다. tf.keras는, 특히 save_weights는 .ckpt 확장자를 가진 텐서플로 [체크포인트](https://www.tensorflow.org/guide/checkpoint?hl=ko) 포맷을 사용합니다(.h5 확장자의 HDF5으로 저장하는 것은 [Save and serialize models 가이드](https://www.tensorflow.org/guide/keras/save_and_serialize?hl=ko#weights_only_saving_in_savedmodel_format)에서 다룹니다):

In [13]:
checkpoint_dir = "/tmp/checkpoints"
print(f"checkpoint_dir={checkpoint_dir}")
if os.path.exists(checkpoint_dir) and os.path.isdir(checkpoint_dir):
    shutil.rmtree(checkpoint_dir)

# 가중치를 저장합니다  
my_checkpoint = os.path.join(checkpoint_dir, 'my_checkpoint')
print(f"my_checkpoint={my_checkpoint}")
model.save_weights(my_checkpoint)

# 새로운 모델 객체를 만듭니다
model = create_model()

# 가중치를 복원합니다
model.load_weights(my_checkpoint)

# 모델을 평가합니다
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

checkpoint_dir=/tmp/checkpoints
my_checkpoint=/tmp/checkpoints/my_checkpoint
32/32 - 0s - loss: 0.3791 - accuracy: 0.8740
복원된 모델의 정확도: 87.40%


## 전체 모델 저장하기

[model.save 메서드](https://www.tensorflow.org/api_docs/python/tf/keras/Model?hl=ko#save)를 호출하여 모델의 구조, 가중치, 훈련 설정을 하나의 파일/폴더에 저장합니다. 모델을 저장하기 때문에 원본 파이썬 코드*가 없어도 사용할 수 있습니다. 옵티마이저 상태가 복원되므로 정확히 중지한 시점에서 다시 훈련을 시작할 수 있습니다.

두 개의 포맷(Saved Model, HDF5)으로 모델을 저장할 수 있습니다. 텐서플로의 SavedModel 포맷은 TF2.x에서기본 파일 포맷입니다. 하지만 HDF5 포맷으로 저장할 수도 있습니다. 두 파일 포맷으로 전체 모델을 저장하는 방법은 아래에서 자세히 설명합니다.

전체 모델을 저장하는 기능은 매우 유용합니다. TensorFlow.js로 모델을 로드한 다음 웹 브라우저에서 모델을 훈련하고 실행할 수 있습니다([SavedModel](https://js.tensorflow.org/tutorials/import-saved-model.html?hl=ko)과 [HDF5](https://js.tensorflow.org/tutorials/import-keras.html?hl=ko)). 또는 모바일 장치에 맞도록 변환한 다음 TensorFlow Lite를 사용하여 실행할 수 있습니다([Saved Model](https://www.tensorflow.org/lite/convert/python_api?hl=ko#exporting_a_savedmodel_), [HDF5](https://www.tensorflow.org/lite/convert/python_api?hl=ko#exporting_a_tfkeras_file_)).

* 사용자 정의 객체(예를 들면 상속으로 만든 클래스나 층)는 저장하고 로드하는데 특별한 주의가 필요합니다. 아래 사용자 정의 객체 저장하기 섹션을 참고하세요.

### SavedModel 포맷

SavedModel 포맷은 모델을 직렬화하는 다른 방법입니다. 이 포맷으로 저장한 모델은 tf.keras.models.load_model로 복원할 수 있고 텐서플로 서빙과 호환됩니다. [SavedModel 가이드](https://www.tensorflow.org/guide/saved_model?hl=ko)에서 SavedModel를 서빙하고 점검하는 자세한 방법을 제공합니다. 이 섹션에서는 모델을 저장하고 복원하는 방법을 안내하겠습니다.

In [14]:
model_dir = "/tmp/models"
print(f"model_dir: {model_dir}")
if os.path.exists(model_dir) and os.path.isdir(model_dir):
    shutil.rmtree(model_dir)

saved_model = os.path.join(model_dir, 'saved_model')
print(f"saved_model={saved_model}")

# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# SavedModel로 전체 모델을 저장합니다
my_model = os.path.join(saved_model, 'my_model')
print(f"my_model={my_model}")
model.save(my_model)

model_dir: /tmp/models
saved_model=/tmp/models/saved_model
Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.6786 - accuracy: 0.4645
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4761 - accuracy: 0.8595
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2819 - accuracy: 0.9307
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2264 - accuracy: 0.9478
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1751 - accuracy: 0.9641
my_model=/tmp/models/saved_model/my_model
INFO:tensorflow:Assets written to: /tmp/models/saved_model/my_model/assets


SavedModel 포맷은 프로토콜 버퍼 이진 파일과 텐서플로 체크포인트를 담고 있는 디렉토리입니다. 저장된 모델의 디렉토리를 확인해 보죠:

In [15]:
# my_model 디렉토리:  assests 폴더, saved_model.pb, variables 폴더
!ls -lR {model_dir}

total 0
drwxr-xr-x  3 comafire  wheel  96  3  7 00:06 saved_model

/tmp/models/saved_model:
total 0
drwxr-xr-x  5 comafire  wheel  160  3  7 00:06 my_model

/tmp/models/saved_model/my_model:
total 168
drwxr-xr-x  2 comafire  wheel     64  3  7 00:06 assets
-rw-r--r--  1 comafire  wheel  82620  3  7 00:06 saved_model.pb
drwxr-xr-x  4 comafire  wheel    128  3  7 00:06 variables

/tmp/models/saved_model/my_model/assets:

/tmp/models/saved_model/my_model/variables:
total 9560
-rw-r--r--  1 comafire  wheel  4888491  3  7 00:06 variables.data-00000-of-00001
-rw-r--r--  1 comafire  wheel     1463  3  7 00:06 variables.index


저장된 모델로부터 새로운 케라스 모델을 로드합니다:



In [16]:
new_model = tf.keras.models.load_model(my_model)

# 모델 구조를 확인합니다
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


복원된 모델은 원본 모델과 동일한 매개변수로 컴파일되어 있습니다. 이 모델을 평가하고 예측에 사용해 보죠:



In [17]:
# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4118 - accuracy: 0.8690
복원된 모델의 정확도: 86.90%
(1000, 10)


### HDF5 파일로 저장하기

케라스는 HDF5 표준을 따르는 기본 저장 포맷을 제공합니다.

In [18]:
model_dir = "/tmp/models"
print(f"model_dir: {model_dir}")
if os.path.exists(model_dir) and os.path.isdir(model_dir):
    shutil.rmtree(model_dir)
    
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
# '.h5' 확장자는 이 모델이 HDF5로 저장되었다는 것을 나타냅니다
my_model = os.path.join(model_dir, 'my_model.h5')
print(f"my_model={my_model}")
model.save(my_model)

model_dir: /tmp/models
Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.5733 - accuracy: 0.5103
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4606 - accuracy: 0.8640
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2970 - accuracy: 0.9265
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2191 - accuracy: 0.9426
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1709 - accuracy: 0.9651
my_model=/tmp/models/my_model.h5


이제 이 파일로부터 모델을 다시 만들어 보죠:


In [19]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
new_model = tf.keras.models.load_model(my_model)

# 모델 구조를 출력합니다
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


정확도를 확인해 보겠습니다:

In [20]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4237 - accuracy: 0.8690
복원된 모델의 정확도: 86.90%


케라스는 모델 구조를 파악하여 저장합니다. 이 방식은 다음과 같은 모든 것을 저장합니다:

* 가중치 값
* 모델 구조
* 모델의 훈련 설정(컴파일할 때 전달한 값)
* 옵티마이저와 필요하면 옵티마이저 상태(중지된 지점부터 다시 시작할 수 있습니다)

체크포인트가 호환되지 않기 때문에 케라스는 v1.x 옵티마이저(tf.compat.v1.train)를 저장할 수 없습니다. v1.x 옵티마이저를 사용하려면 로드한 후에 모델을 다시 컴파일해야 합니다. 따라서 옵티마이저의 상태를 잃게 됩니다.

### 사용자 정의 객체

SaveModel 포맷을 사용한다면 이 섹션은 건너 뛰어도 됩니다. HDF5와 SavedModel 사이의 주요 차이점은 HDF5가 객체로 모델 구조를 저장하고 SavedModel은 실행 그래프로 저장하는 것입니다. 따라서 SavedModel은 원본 코드 없이 상속 모델과 사용자 정의 층 같은 사용자 정의 객체를 저장할 수 있습니다.

사용자 정의 객체를 HDF5로 저장하려면 다음 과정을 따르세요:

* 이 객체에 get_config 메서드를 정의하고 선택적으로 from_config 클래스 메서드를 정의합니다.
 * get_config(self)는 객체를 다시 생성하기 위해 필요한 JSON 직렬화된 매개변수 딕셔너리를 반환합니다.
 * from_config(cls, config)는 get_config에서 반환된 설정을 사용해 새로운 객체를 만듭니다. 기본적으로 이 함수는 이 설정을 초기화 메서드의 매개변수로 사용합니다(return cls(**config)).
* 모델을 로드할 때 이 객체를 custom_objects 매개변수로 전달합니다. 문자열 클래스 이름과 파이썬 클래스를 매핑한 딕서너리를 매개변수로 제공해야 합니다. 예를 들면 tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})

사용자 정의 객체와 get_config에 관한 예제를 보려면 [Writing layers and models from scratch 튜토리얼](https://www.tensorflow.org/guide/keras/custom_layers_and_models?hl=ko)을 참고하세요.

In [21]:
# MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.